In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['dccuchile/bert-base-spanish-wwm-cased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

...

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.23299162089824677
DEV ACC: 0.8211897413286302
DEV Precision: 0.664037769990151
DEV Recall: 0.8044532627865961
DEV F1Score: 0.709426146854276
BEST ACCURACY -->  DEV: 0.82119
BEST PRECISION -->  DEV: 0.66404
BEST RECALL -->  DEV: 0.80445
BEST F1SCORE -->  DEV: 0.70943
TIME PER EPOCH: 6.641832133134206

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHE

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.3138262629508972
DEV ACC: 0.826113315696649
DEV Precision: 0.617766140385188
DEV Recall: 0.8880889392794155
DEV F1Score: 0.7148518008768056
BEST ACCURACY -->  DEV: 0.82611
BEST PRECISION -->  DEV: 0.61777
BEST RECALL -->  DEV: 0.88809
BEST F1SCORE -->  DEV: 0.71485
TIME PER EPOCH: 6.694891615708669

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Training epoch: 1
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED BATCH: 239 of 252
Validation loss per 100 evaluation steps: 0.438001424074173
DEV ACC: 0.8251212522045855
DEV Precision: 0.6614008566389518
DEV Recall: 0.8621000251952631
DEV F1Score: 0.7307000500277808
BEST ACCURACY -->  DEV: 0.82512
BEST PRECISION -->  DEV: 0.6614
BEST RECALL -->  DEV: 0.8621
BEST F1SCORE -->  DEV: 0.7307
TIME PER EPOCH: 6.642823914686839

Training epoch: 2
FINSIHED BATCH: 19 of 252
FINSIHED BATCH: 39 of 252
FINSIHED BATCH: 59 of 252
FINSIHED BATCH: 79 of 252
FINSIHED BATCH: 99 of 252
FINSIHED BATCH: 119 of 252
FINSIHED BATCH: 139 of 252
FINSIHED BATCH: 159 of 252
FINSIHED BATCH: 179 of 252
FINSIHED BATCH: 199 of 252
FINSIHED BATCH: 219 of 252
FINSIHED B

In [6]:
print(best_dev_acc)

0.8251212522045855


In [7]:
print(best_epoch)

0


In [8]:
print(best_dev_precision)

0.7361477587668064


In [9]:
print(best_dev_recall)

0.8621000251952631


In [10]:
print(best_dev_f1score)

0.7307000500277808


In [11]:
print(overall_list_dev_acc)

[[0.8211529982363315, 0.8240373309817755, 0.8364381246325692, 0.8329659024103468, 0.8075764256319812, 0.8074845679012346, 0.7939998530276308, 0.813749265138154, 0.8142085537918871, 0.814024838330394], [0.8166887125220458, 0.8334619341563786, 0.8248456790123457, 0.8188014403292181, 0.819077013521458, 0.8152924750146971, 0.8182686654908877, 0.817092886537331, 0.820160934744268, 0.822769694297472], [0.8211897413286302, 0.8373383303938859, 0.8343253968253969, 0.8238536155202822, 0.8134369488536155, 0.8198853615520282, 0.8114528218694885, 0.8186728395061728, 0.7979313639035861, 0.8095605526161082], [0.826113315696649, 0.8348214285714286, 0.8283179012345679, 0.8249375367430923, 0.8218694885361553, 0.8171847442680775, 0.8035163139329806, 0.8129041740152851, 0.8229534097589654, 0.8107363315696648], [0.8251212522045855, 0.8171296296296295, 0.8251212522045855, 0.8114528218694885, 0.824349647266314, 0.8114528218694885, 0.8196649029982362, 0.8090277777777778, 0.814024838330394, 0.8094686948853616]

In [12]:
print(overall_list_dev_precision)

[[0.6447848447848448, 0.720880574452003, 0.7079977782358733, 0.6774158821777871, 0.6611489040060469, 0.6706269039602373, 0.661904761904762, 0.6213220641792072, 0.6612743077028793, 0.7077097505668936], [0.6134411884411883, 0.7102922650541698, 0.6979591836734693, 0.7334278155706726, 0.6945137314184932, 0.7007558578987151, 0.6517476350809681, 0.6920800980324788, 0.61014171014171, 0.698681124871601], [0.664037769990151, 0.6728074394741062, 0.6565747303842542, 0.6545414462081128, 0.707734945830184, 0.6205152431342907, 0.6365651984699603, 0.6858952335142809, 0.6502427907189814, 0.6487322201607916], [0.617766140385188, 0.6566045946998327, 0.6985701688082642, 0.6407395955015002, 0.7011360773265536, 0.6699575116241783, 0.646182917611489, 0.7102418745275887, 0.6320970475732383, 0.6667439703153989], [0.6614008566389518, 0.7361477587668064, 0.6497606449987402, 0.658390022675737, 0.641361372313753, 0.6468379944570422, 0.6536739274834513, 0.6943373645754599, 0.675009161913924, 0.6730100146766815]]


In [13]:
print(overall_list_dev_recall)

[[0.8407911312673219, 0.578649304839781, 0.7326954350763873, 0.7907533383723858, 0.48360418241370623, 0.6172461577223479, 0.4968603266222314, 0.8244313887171029, 0.7424540186444948, 0.5972193591241212], [0.9083390595295356, 0.7138259007306628, 0.7734115531734579, 0.5696586041824138, 0.6354943997801139, 0.6323530085434849, 0.6644775418584943, 0.641777754872993, 0.8540690350214161, 0.7584530108339633], [0.8044532627865961, 0.8123330813807003, 0.8224048878810782, 0.7226190476190476, 0.5849080372889898, 0.7902431342907531, 0.8195389266817839, 0.7151400627591102, 0.6236171236171236, 0.667358391167915], [0.8880889392794155, 0.8537666918619299, 0.6942554799697659, 0.845419615657711, 0.7087805492567397, 0.6495705352848212, 0.5770133305847595, 0.5615772234819855, 0.707353581163105, 0.7318891866510912], [0.8621000251952631, 0.6373479694908266, 0.6848628003389908, 0.610424540186445, 0.7928445452254974, 0.6852607709750566, 0.7479969765684051, 0.5603111615016376, 0.6791761148904006, 0.6343188997950

In [14]:
print(overall_list_dev_f1score)

[[0.7117899252832173, 0.6176725816486984, 0.6966230083140004, 0.7110809645086513, 0.5243058891518275, 0.618675446295968, 0.5405893455134205, 0.6861110523354367, 0.6841462793217178, 0.6279014869351005], [0.7176351282806677, 0.679046047323358, 0.7119038835425389, 0.6149821285675626, 0.6334544980903524, 0.6446763450264851, 0.6417374406955134, 0.6358780989733369, 0.693054270948655, 0.7049854434978048], [0.709426146854276, 0.7184108399831528, 0.7153883374803323, 0.6630780577338148, 0.6186918460727985, 0.6785681914091171, 0.7004849811351359, 0.6724744290496906, 0.613492208590248, 0.6422449891637567], [0.7148518008768056, 0.726783250003064, 0.6763995808113454, 0.7159270988851332, 0.6758592273998435, 0.631712951951047, 0.5782836548242711, 0.6046669380002712, 0.6487424464462647, 0.6746135421684586], [0.7307000500277808, 0.6534113386354283, 0.6465418002422276, 0.6148914229741298, 0.6911905032463778, 0.6317172849864873, 0.6768146017114022, 0.5928113125592117, 0.6478080817790385, 0.625075489358158

In [15]:
#Best model is model 4